In [1]:
import sys
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import optimizers
from keras import regularizers
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support

%matplotlib inline

print(keras.__version__)

Using TensorFlow backend.


Instructions for updating:
Use the retry module or similar alternatives.
2.0.8


In [2]:
NORM = 'stdnorm'
DISTRIBUTION = 7
FILLING = 'cero2mean'

LABELS_OUT_FILE = './datasets/labels-whole-t' + str(DISTRIBUTION) + '-' + FILLING + '-' + NORM + '.npy'
IMAGES_OUT_FILE = './datasets/images-whole-t' + str(DISTRIBUTION) + '-' + FILLING + '-' + NORM + '.npy'

FINGERS = 3
TACTILE_IMAGE_ROWS = 12
TACTILE_IMAGE_COLS = 11

In [3]:
labels = np.load(LABELS_OUT_FILE)
labels_cat = np_utils.to_categorical(labels, num_classes=2)
tactile_images = np.load(IMAGES_OUT_FILE)

In [4]:
print(tactile_images[0])

[[[ 0.20416431  0.20416431  0.48778092  0.77139752  0.79018008
    0.80896263  0.73852805  0.66809346  0.27647716 -0.11513915
   -0.11513915]
  [ 0.20416431  0.48778092  0.77139752  0.77139752  0.80896263
    0.80896263  0.80896263  0.66809346  0.66809346  0.27647716
   -0.11513915]
  [ 1.25260662  0.77139752  0.77139752  0.77515403  0.79393659
    0.80896263  0.72256287  0.65212829  0.66809346  0.66809346
    0.12133324]
  [ 2.39984517  2.39984517  1.11429657  0.77891054  0.77891054
    0.74134543  0.63616311  0.63616311  0.39750725 -0.11138264
   -0.11138264]
  [ 2.39984517  1.67953413  0.95922309  0.77891054  0.58075457
    0.59922409  0.50938086  0.63616311  0.05766037 -0.02686113
   -0.11138264]
  [ 1.67953413  0.95922309  0.62301532  0.28680756  0.33470308
    0.3825986  -0.28793867 -0.95847593 -0.45040778  0.05766037
   -0.02686113]
  [ 0.43143325  0.43143325  0.62301532  0.28680756  0.33470308
    0.3825986  -0.28793867 -0.95847593 -0.45040778 -0.08226967
   -0.08226967]
  [-0.

# CNN1: 32 conv 3x3 - 1024 fc

In [5]:
folds = 10
kfold = StratifiedKFold(n_splits=folds, shuffle=True)
cv_accuracies = []
cv_precision = []
cv_recall = []
cv_f1_score = []

for index, (train_indices, val_indices) in enumerate(kfold.split(tactile_images, labels)):
    print("Training fold " + str(index + 1) + "/" + str(folds))

    # split data
    images_train, images_val = tactile_images[train_indices], tactile_images[val_indices]
    labels_train, labels_val = labels_cat[train_indices], labels_cat[val_indices]  
    
    # build model
    epochs = 50
    batch = 32
        
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(FINGERS, TACTILE_IMAGE_ROWS, TACTILE_IMAGE_COLS), 
                     data_format='channels_first'))
    model.add(Activation('relu'))
    
    model.add(Flatten())
    
    model.add(Dense(1024))
    model.add(Activation('relu'))
    
    model.add(Dense(2, activation='softmax'))

    # compile and train
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=0.0001),
                  metrics=['accuracy'])
    model.fit(images_train, labels_train, epochs=epochs, batch_size=batch, verbose=1)
    
    # evaluate
    print("# # # Evaluating cv-fold...")
    scores = model.evaluate(images_val, labels_val, verbose=1)
    cv_accuracies.append(scores[1] * 100)
    
    predictions = model.predict(images_val)
    [precision, recall, f1_score, _] = precision_recall_fscore_support(np.argmax(labels_val, axis=1), 
                                                                          np.argmax(predictions, axis=1), 
                                                                          average='binary', pos_label=1)
    
    cv_precision.append(precision * 100)
    cv_recall.append(recall * 100)
    cv_f1_score.append(f1_score * 100)
    
    print("\naccuracy:", scores[1] * 100)
    print("precision:", precision * 100)
    print("recall:", recall * 100)
    print("f1_score:", f1_score * 100)
    
print("\nFinal cross-validation score:")
print(np.mean(cv_accuracies), "+/-", np.std(cv_accuracies))
print(np.mean(cv_precision), "+/-", np.std(cv_precision))
print(np.mean(cv_recall), "+/-", np.std(cv_recall))
print(np.mean(cv_f1_score), "+/-", np.std(cv_f1_score))

# cer2lesscontact - D7
#91.8792648371013 +/- 1.2524200440715512
#91.47150211573612 +/- 2.7030363900110417
#92.57012795275591 +/- 1.9644354641022856
#91.97094804781125 +/- 1.1495731346986442

Training fold 1/10
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/50
2294/2294 [==============================] - 3s - loss: 0.3827 - acc: 0.8274     
Epoch 2/50
2294/2294 [==============================] - 0s - loss: 0.3221 - acc: 0.8605     
Epoch 3/50
2294/2294 [==============================] - 0s - loss: 0.2703 - acc: 0.8849     
Epoch 4/50
2294/2294 [==============================] - 0s - loss: 0.2538 - acc: 0.8915     
Epoch 5/50
2294/2294 [==============================] - 0s - loss: 0.2434 - acc: 0.8971     
Epoch 6/50
2294/2294 [==============================] - 0s - loss: 0.2413 - acc: 0.9037     
Epoch 7/50
2294/2294 [==============================] - 0s - loss: 0.2204 - acc: 0.9154     
Epoch 8/50
2294/2294 [==============================] - 0s - loss: 0.2168 - acc: 0.9111     
Epoch 9/50
2294/22

2294/2294 [==============================] - 0s - loss: 0.1183 - acc: 0.9534     
Epoch 26/50
2294/2294 [==============================] - 0s - loss: 0.1147 - acc: 0.9608     
Epoch 27/50
2294/2294 [==============================] - 0s - loss: 0.1101 - acc: 0.9629     
Epoch 28/50
2294/2294 [==============================] - 0s - loss: 0.1154 - acc: 0.9568     
Epoch 29/50
2294/2294 [==============================] - 0s - loss: 0.1032 - acc: 0.9616     
Epoch 30/50
2294/2294 [==============================] - 0s - loss: 0.1082 - acc: 0.9634     
Epoch 31/50
2294/2294 [==============================] - 0s - loss: 0.1050 - acc: 0.9634     
Epoch 32/50
2294/2294 [==============================] - 0s - loss: 0.0983 - acc: 0.9677     
Epoch 33/50
2294/2294 [==============================] - 0s - loss: 0.1002 - acc: 0.9612     
Epoch 34/50
2294/2294 [==============================] - 0s - loss: 0.0885 - acc: 0.9690     
Epoch 35/50
2294/2294 [==============================] - 0s - loss: 0.08

2294/2294 [==============================] - 0s - loss: 0.2116 - acc: 0.9167     
Epoch 9/50
2294/2294 [==============================] - 0s - loss: 0.2058 - acc: 0.9194     
Epoch 10/50
2294/2294 [==============================] - 0s - loss: 0.2002 - acc: 0.9194     
Epoch 11/50
2294/2294 [==============================] - 0s - loss: 0.1871 - acc: 0.9276     
Epoch 12/50
2294/2294 [==============================] - 0s - loss: 0.1728 - acc: 0.9403     
Epoch 13/50
2294/2294 [==============================] - 0s - loss: 0.1699 - acc: 0.9377     
Epoch 14/50
2294/2294 [==============================] - 0s - loss: 0.1649 - acc: 0.9372     
Epoch 15/50
2294/2294 [==============================] - 0s - loss: 0.1598 - acc: 0.9394     
Epoch 16/50
2294/2294 [==============================] - 0s - loss: 0.1514 - acc: 0.9433     
Epoch 17/50
2294/2294 [==============================] - 0s - loss: 0.1517 - acc: 0.9429     
Epoch 18/50
2294/2294 [==============================] - 0s - loss: 0.137

2294/2294 [==============================] - 0s - loss: 0.0726 - acc: 0.9752     
Epoch 43/50
2294/2294 [==============================] - 0s - loss: 0.0655 - acc: 0.9795     
Epoch 44/50
2294/2294 [==============================] - 0s - loss: 0.0692 - acc: 0.9760     
Epoch 45/50
2294/2294 [==============================] - 0s - loss: 0.0612 - acc: 0.9817     
Epoch 46/50
2294/2294 [==============================] - 0s - loss: 0.0582 - acc: 0.9843     
Epoch 47/50
2294/2294 [==============================] - 0s - loss: 0.0558 - acc: 0.9856     
Epoch 48/50
2294/2294 [==============================] - 0s - loss: 0.0567 - acc: 0.9826     
Epoch 49/50
2294/2294 [==============================] - 0s - loss: 0.0529 - acc: 0.9869     
Epoch 50/50
2294/2294 [==============================] - 0s - loss: 0.0520 - acc: 0.9856     
# # # Evaluating cv-fold...
 32/255 [==>...........................] - ETA: 0s
accuracy: 91.76470555511176
precision: 91.47286821705426
recall: 92.1875
f1_score: 91.8

2294/2294 [==============================] - 0s - loss: 0.1122 - acc: 0.9586     
Epoch 25/50
2294/2294 [==============================] - 0s - loss: 0.1101 - acc: 0.9634     
Epoch 26/50
2294/2294 [==============================] - 0s - loss: 0.1065 - acc: 0.9616     
Epoch 27/50
2294/2294 [==============================] - 0s - loss: 0.1021 - acc: 0.9677     
Epoch 28/50
2294/2294 [==============================] - 0s - loss: 0.0957 - acc: 0.9669     
Epoch 29/50
2294/2294 [==============================] - 0s - loss: 0.0990 - acc: 0.9677     
Epoch 30/50
2294/2294 [==============================] - 0s - loss: 0.0974 - acc: 0.9717     
Epoch 31/50
2294/2294 [==============================] - 0s - loss: 0.0986 - acc: 0.9660     
Epoch 32/50
2294/2294 [==============================] - 0s - loss: 0.0834 - acc: 0.9734     
Epoch 33/50
2294/2294 [==============================] - 0s - loss: 0.0959 - acc: 0.9660     
Epoch 34/50
2294/2294 [==============================] - 0s - loss: 0.08

2294/2294 [==============================] - 0s - loss: 0.2171 - acc: 0.9194     
Epoch 8/50
2294/2294 [==============================] - 0s - loss: 0.2030 - acc: 0.9263     
Epoch 9/50
2294/2294 [==============================] - 0s - loss: 0.1936 - acc: 0.9255     
Epoch 10/50
2294/2294 [==============================] - 0s - loss: 0.1830 - acc: 0.9324     
Epoch 11/50
2294/2294 [==============================] - 0s - loss: 0.1816 - acc: 0.9337     
Epoch 12/50
2294/2294 [==============================] - 0s - loss: 0.1692 - acc: 0.9398     
Epoch 13/50
2294/2294 [==============================] - 0s - loss: 0.1575 - acc: 0.9416     
Epoch 14/50
2294/2294 [==============================] - 0s - loss: 0.1581 - acc: 0.9433     
Epoch 15/50
2294/2294 [==============================] - 0s - loss: 0.1538 - acc: 0.9446     
Epoch 16/50
2294/2294 [==============================] - 0s - loss: 0.1440 - acc: 0.9503     
Epoch 17/50
2294/2294 [==============================] - 0s - loss: 0.1353

2295/2295 [==============================] - 0s - loss: 0.0647 - acc: 0.9843     
Epoch 43/50
2295/2295 [==============================] - 0s - loss: 0.0598 - acc: 0.9834     
Epoch 44/50
2295/2295 [==============================] - 0s - loss: 0.0597 - acc: 0.9795     
Epoch 45/50
2295/2295 [==============================] - 0s - loss: 0.0611 - acc: 0.9813     
Epoch 46/50
2295/2295 [==============================] - 0s - loss: 0.0620 - acc: 0.9804     
Epoch 47/50
2295/2295 [==============================] - 0s - loss: 0.0578 - acc: 0.9843     
Epoch 48/50
2295/2295 [==============================] - 0s - loss: 0.0592 - acc: 0.9834     
Epoch 49/50
2295/2295 [==============================] - 0s - loss: 0.0568 - acc: 0.9830     
Epoch 50/50
2295/2295 [==============================] - 0s - loss: 0.0609 - acc: 0.9773     
# # # Evaluating cv-fold...
 32/254 [==>...........................] - ETA: 0s
accuracy: 92.12598434583408
precision: 94.9579831932773
recall: 88.9763779527559
f1_sco

# CNN2: 32 conv 3x3 - max pool 2x2 - 1024 fc

In [6]:
folds = 10
kfold = StratifiedKFold(n_splits=folds, shuffle=True)
cv_accuracies = []
cv_precision = []
cv_recall = []
cv_f1_score = []

for index, (train_indices, val_indices) in enumerate(kfold.split(tactile_images, labels)):
    print("Training fold " + str(index + 1) + "/" + str(folds))

    # split data
    images_train, images_val = tactile_images[train_indices], tactile_images[val_indices]
    labels_train, labels_val = labels_cat[train_indices], labels_cat[val_indices]  
    
    # build model
    epochs = 50
    batch = 32
        
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(FINGERS, TACTILE_IMAGE_ROWS, TACTILE_IMAGE_COLS), 
                     data_format='channels_first'))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(2,2), data_format='channels_first'))
    
    model.add(Flatten())
    
    model.add(Dense(1024))
    model.add(Activation('relu'))
    
    model.add(Dense(2, activation='softmax'))

    # compile and train
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=0.0001),
                  metrics=['accuracy'])
    model.fit(images_train, labels_train, epochs=epochs, batch_size=batch, verbose=1)
    
    # evaluate
    print("# # # Evaluating cv-fold...")
    scores = model.evaluate(images_val, labels_val, verbose=1)
    cv_accuracies.append(scores[1] * 100)
    
    predictions = model.predict(images_val)
    [precision, recall, f1_score, _] = precision_recall_fscore_support(np.argmax(labels_val, axis=1), 
                                                                          np.argmax(predictions, axis=1), 
                                                                          average='binary', pos_label=1)
    
    cv_precision.append(precision * 100)
    cv_recall.append(recall * 100)
    cv_f1_score.append(f1_score * 100)
    
    print("\naccuracy:", scores[1] * 100)
    print("precision:", precision * 100)
    print("recall:", recall * 100)
    print("f1_score:", f1_score * 100)
    
print("\nFinal cross-validation score:")
print(np.mean(cv_accuracies), "+/-", np.std(cv_accuracies))
print(np.mean(cv_precision), "+/-", np.std(cv_precision))
print(np.mean(cv_recall), "+/-", np.std(cv_recall))
print(np.mean(cv_f1_score), "+/-", np.std(cv_f1_score))

# IROS
#89.6039832747 +/- 2.41809610008
#90.6084867258 +/- 2.90308891937
#88.5851377953 +/- 4.33772241807
#89.4995478537 +/- 2.49870737736

# cero2lesscontact - D7:
#90.42674068146067 +/- 1.3108566178983423
#91.28939662477264 +/- 2.1046859583458124
#89.52263779527559 +/- 1.8902895538940776
#90.3709595238914 +/- 1.3055870466254564

Training fold 1/10
Epoch 1/50
2294/2294 [==============================] - 1s - loss: 0.4461 - acc: 0.8012     
Epoch 2/50
2294/2294 [==============================] - 0s - loss: 0.3503 - acc: 0.8527     
Epoch 3/50
2294/2294 [==============================] - 0s - loss: 0.3267 - acc: 0.8618     
Epoch 4/50
2294/2294 [==============================] - 0s - loss: 0.3051 - acc: 0.8718     
Epoch 5/50
2294/2294 [==============================] - 0s - loss: 0.2893 - acc: 0.8758     
Epoch 6/50
2294/2294 [==============================] - 0s - loss: 0.2764 - acc: 0.8849     
Epoch 7/50
2294/2294 [==============================] - 0s - loss: 0.2717 - acc: 0.8875     
Epoch 8/50
2294/2294 [==============================] - 0s - loss: 0.2586 - acc: 0.8915     
Epoch 9/50
2294/2294 [==============================] - 0s - loss: 0.2480 - acc: 0.9002     
Epoch 10/50
2294/2294 [==============================] - 0s - loss: 0.2424 - acc: 0.8993     
Epoch 11/50
2294/2294 [===========================

2294/2294 [==============================] - 0s - loss: 0.1494 - acc: 0.9490     
Epoch 37/50
2294/2294 [==============================] - 0s - loss: 0.1496 - acc: 0.9477     
Epoch 38/50
2294/2294 [==============================] - 0s - loss: 0.1466 - acc: 0.9516     
Epoch 39/50
2294/2294 [==============================] - 0s - loss: 0.1464 - acc: 0.9490     
Epoch 40/50
2294/2294 [==============================] - 0s - loss: 0.1412 - acc: 0.9551     
Epoch 41/50
2294/2294 [==============================] - 0s - loss: 0.1490 - acc: 0.9459     
Epoch 42/50
2294/2294 [==============================] - 0s - loss: 0.1345 - acc: 0.9573     
Epoch 43/50
2294/2294 [==============================] - 0s - loss: 0.1340 - acc: 0.9555     
Epoch 44/50
2294/2294 [==============================] - 0s - loss: 0.1293 - acc: 0.9608     
Epoch 45/50
2294/2294 [==============================] - 0s - loss: 0.1288 - acc: 0.9582     
Epoch 46/50
2294/2294 [==============================] - 0s - loss: 0.12

2294/2294 [==============================] - 0s - loss: 0.1964 - acc: 0.9276     
Epoch 20/50
2294/2294 [==============================] - 0s - loss: 0.1909 - acc: 0.9298     
Epoch 21/50
2294/2294 [==============================] - 0s - loss: 0.1867 - acc: 0.9307     
Epoch 22/50
2294/2294 [==============================] - 0s - loss: 0.1889 - acc: 0.9259     
Epoch 23/50
2294/2294 [==============================] - 0s - loss: 0.1845 - acc: 0.9285     
Epoch 24/50
2294/2294 [==============================] - 0s - loss: 0.1773 - acc: 0.9398     
Epoch 25/50
2294/2294 [==============================] - 0s - loss: 0.1701 - acc: 0.9429     
Epoch 26/50
2294/2294 [==============================] - 0s - loss: 0.1669 - acc: 0.9390     
Epoch 27/50
2294/2294 [==============================] - 0s - loss: 0.1675 - acc: 0.9368     
Epoch 28/50
2294/2294 [==============================] - 0s - loss: 0.1642 - acc: 0.9429     
Epoch 29/50
2294/2294 [==============================] - 0s - loss: 0.15

2294/2294 [==============================] - 0s - loss: 0.3564 - acc: 0.8496     
Epoch 3/50
2294/2294 [==============================] - 0s - loss: 0.3256 - acc: 0.8596     
Epoch 4/50
2294/2294 [==============================] - 0s - loss: 0.3038 - acc: 0.8701     
Epoch 5/50
2294/2294 [==============================] - 0s - loss: 0.2915 - acc: 0.8823     
Epoch 6/50
2294/2294 [==============================] - 0s - loss: 0.2805 - acc: 0.8827     
Epoch 7/50
2294/2294 [==============================] - 0s - loss: 0.2733 - acc: 0.8871     
Epoch 8/50
2294/2294 [==============================] - 0s - loss: 0.2581 - acc: 0.8915     
Epoch 9/50
2294/2294 [==============================] - 0s - loss: 0.2514 - acc: 0.9006     
Epoch 10/50
2294/2294 [==============================] - 0s - loss: 0.2472 - acc: 0.9024     
Epoch 11/50
2294/2294 [==============================] - 0s - loss: 0.2366 - acc: 0.9115     
Epoch 12/50
2294/2294 [==============================] - 0s - loss: 0.2373 - ac

2294/2294 [==============================] - 0s - loss: 0.1457 - acc: 0.9516     
Epoch 38/50
2294/2294 [==============================] - 0s - loss: 0.1403 - acc: 0.9516     
Epoch 39/50
2294/2294 [==============================] - 0s - loss: 0.1422 - acc: 0.9512     
Epoch 40/50
2294/2294 [==============================] - 0s - loss: 0.1411 - acc: 0.9503     
Epoch 41/50
2294/2294 [==============================] - 0s - loss: 0.1397 - acc: 0.9534     
Epoch 42/50
2294/2294 [==============================] - 0s - loss: 0.1358 - acc: 0.9555     
Epoch 43/50
2294/2294 [==============================] - 0s - loss: 0.1320 - acc: 0.9529     
Epoch 44/50
2294/2294 [==============================] - 0s - loss: 0.1297 - acc: 0.9573     
Epoch 45/50
2294/2294 [==============================] - 0s - loss: 0.1313 - acc: 0.9555     
Epoch 46/50
2294/2294 [==============================] - 0s - loss: 0.1248 - acc: 0.9603     
Epoch 47/50
2294/2294 [==============================] - 0s - loss: 0.13

2294/2294 [==============================] - 0s - loss: 0.1962 - acc: 0.9272     
Epoch 21/50
2294/2294 [==============================] - 0s - loss: 0.1931 - acc: 0.9307     
Epoch 22/50
2294/2294 [==============================] - 0s - loss: 0.1872 - acc: 0.9342     
Epoch 23/50
2294/2294 [==============================] - 0s - loss: 0.1886 - acc: 0.9289     
Epoch 24/50
2294/2294 [==============================] - 0s - loss: 0.1836 - acc: 0.9329     
Epoch 25/50
2294/2294 [==============================] - 0s - loss: 0.1825 - acc: 0.9316     
Epoch 26/50
2294/2294 [==============================] - 0s - loss: 0.1782 - acc: 0.9342     
Epoch 27/50
2294/2294 [==============================] - 0s - loss: 0.1709 - acc: 0.9346     
Epoch 28/50
2294/2294 [==============================] - 0s - loss: 0.1695 - acc: 0.9377     
Epoch 29/50
2294/2294 [==============================] - 0s - loss: 0.1607 - acc: 0.9433     
Epoch 30/50
2294/2294 [==============================] - 0s - loss: 0.16

# CNN3: 32 conv 3x3 - max pool 2x2 - 64 conv 1x1 - 1024 fc

In [7]:
folds = 10
kfold = StratifiedKFold(n_splits=folds, shuffle=True)
cv_accuracies = []
cv_precision = []
cv_recall = []
cv_f1_score = []

for index, (train_indices, val_indices) in enumerate(kfold.split(tactile_images, labels)):
    print("Training fold " + str(index + 1) + "/" + str(folds))

    # split data
    images_train, images_val = tactile_images[train_indices], tactile_images[val_indices]
    labels_train, labels_val = labels_cat[train_indices], labels_cat[val_indices]  
    
    # build model
    epochs = 50
    batch = 32
        
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(FINGERS, TACTILE_IMAGE_ROWS, TACTILE_IMAGE_COLS), 
                     data_format='channels_first'))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(2,2), data_format='channels_first'))
    
    model.add(Conv2D(64, (1, 1), data_format='channels_first'))
    model.add(Activation('relu'))
    
    model.add(Flatten())
    
    model.add(Dense(1024))
    model.add(Activation('relu'))
    
    model.add(Dense(2, activation='softmax'))

    # compile and train
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=0.0001),
                  metrics=['accuracy'])
    model.fit(images_train, labels_train, epochs=epochs, batch_size=batch, verbose=1)
    
    # evaluate
    print("# # # Evaluating cv-fold...")
    scores = model.evaluate(images_val, labels_val, verbose=1)
    cv_accuracies.append(scores[1] * 100)
    
    predictions = model.predict(images_val)
    [precision, recall, f1_score, _] = precision_recall_fscore_support(np.argmax(labels_val, axis=1), 
                                                                          np.argmax(predictions, axis=1), 
                                                                          average='binary', pos_label=1)
    
    cv_precision.append(precision * 100)
    cv_recall.append(recall * 100)
    cv_f1_score.append(f1_score * 100)
    
    print("\naccuracy:", scores[1] * 100)
    print("precision:", precision * 100)
    print("recall:", recall * 100)
    print("f1_score:", f1_score * 100)
    
print("\nFinal cross-validation score:")
print(np.mean(cv_accuracies), "+/-", np.std(cv_accuracies))
print(np.mean(cv_precision), "+/-", np.std(cv_precision))
print(np.mean(cv_recall), "+/-", np.std(cv_recall))
print(np.mean(cv_f1_score), "+/-", np.std(cv_f1_score))

# IROS
#90.3493898063 +/- 1.12377139854
#91.4068506889 +/- 2.17906727927
#89.2882627953 +/- 3.73186597324
#90.2480793929 +/- 1.33624350767

# cero2lesscontact - D7
#91.40759604389844 +/- 1.2767366561486764
#92.67978058788526 +/- 3.7628189660204914
#90.30819389763779 +/- 3.8417126413849445
#91.33575532687321 +/- 1.2742690190665134

Training fold 1/10
Epoch 1/50
2294/2294 [==============================] - 0s - loss: 0.4704 - acc: 0.7956     
Epoch 2/50
2294/2294 [==============================] - 0s - loss: 0.3626 - acc: 0.8461     
Epoch 3/50
2294/2294 [==============================] - 0s - loss: 0.3282 - acc: 0.8618     
Epoch 4/50
2294/2294 [==============================] - 0s - loss: 0.3011 - acc: 0.8784     
Epoch 5/50
2294/2294 [==============================] - 0s - loss: 0.2881 - acc: 0.8819     
Epoch 6/50
2294/2294 [==============================] - 0s - loss: 0.2738 - acc: 0.8858     
Epoch 7/50
2294/2294 [==============================] - 0s - loss: 0.2594 - acc: 0.8980     
Epoch 8/50
2294/2294 [==============================] - 0s - loss: 0.2547 - acc: 0.8919     
Epoch 9/50
2294/2294 [==============================] - 0s - loss: 0.2391 - acc: 0.9058     
Epoch 10/50
2294/2294 [==============================] - 0s - loss: 0.2291 - acc: 0.9050     
Epoch 11/50
2294/2294 [===========================

2294/2294 [==============================] - 0s - loss: 0.1311 - acc: 0.9568     
Epoch 37/50
2294/2294 [==============================] - 0s - loss: 0.1264 - acc: 0.9582     
Epoch 38/50
2294/2294 [==============================] - 0s - loss: 0.1268 - acc: 0.9564     
Epoch 39/50
2294/2294 [==============================] - 0s - loss: 0.1199 - acc: 0.9625     
Epoch 40/50
2294/2294 [==============================] - 0s - loss: 0.1214 - acc: 0.9603     
Epoch 41/50
2294/2294 [==============================] - 0s - loss: 0.1221 - acc: 0.9586     
Epoch 42/50
2294/2294 [==============================] - 0s - loss: 0.1233 - acc: 0.9586     
Epoch 43/50
2294/2294 [==============================] - 0s - loss: 0.1133 - acc: 0.9634     
Epoch 44/50
2294/2294 [==============================] - 0s - loss: 0.1069 - acc: 0.9690     
Epoch 45/50
2294/2294 [==============================] - 0s - loss: 0.1087 - acc: 0.9673     
Epoch 46/50
2294/2294 [==============================] - 0s - loss: 0.10

2294/2294 [==============================] - 0s - loss: 0.1725 - acc: 0.9320     
Epoch 20/50
2294/2294 [==============================] - 0s - loss: 0.1722 - acc: 0.9350     
Epoch 21/50
2294/2294 [==============================] - 0s - loss: 0.1675 - acc: 0.9368     
Epoch 22/50
2294/2294 [==============================] - 0s - loss: 0.1691 - acc: 0.9390     
Epoch 23/50
2294/2294 [==============================] - 0s - loss: 0.1588 - acc: 0.9438     
Epoch 24/50
2294/2294 [==============================] - 0s - loss: 0.1576 - acc: 0.9398     
Epoch 25/50
2294/2294 [==============================] - 0s - loss: 0.1467 - acc: 0.9468     
Epoch 26/50
2294/2294 [==============================] - 0s - loss: 0.1474 - acc: 0.9481     
Epoch 27/50
2294/2294 [==============================] - 0s - loss: 0.1467 - acc: 0.9459     
Epoch 28/50
2294/2294 [==============================] - 0s - loss: 0.1386 - acc: 0.9503     
Epoch 29/50
2294/2294 [==============================] - 0s - loss: 0.13

2294/2294 [==============================] - 0s - loss: 0.3468 - acc: 0.8492     
Epoch 3/50
2294/2294 [==============================] - 0s - loss: 0.3174 - acc: 0.8653     
Epoch 4/50
2294/2294 [==============================] - 0s - loss: 0.2959 - acc: 0.8723     
Epoch 5/50
2294/2294 [==============================] - 0s - loss: 0.2901 - acc: 0.8753     
Epoch 6/50
2294/2294 [==============================] - 0s - loss: 0.2702 - acc: 0.8915     
Epoch 7/50
2294/2294 [==============================] - 0s - loss: 0.2610 - acc: 0.8954     
Epoch 8/50
2294/2294 [==============================] - 0s - loss: 0.2508 - acc: 0.8936     
Epoch 9/50
2294/2294 [==============================] - 0s - loss: 0.2462 - acc: 0.9019     
Epoch 10/50
2294/2294 [==============================] - 0s - loss: 0.2354 - acc: 0.9054     
Epoch 11/50
2294/2294 [==============================] - 0s - loss: 0.2379 - acc: 0.9080     
Epoch 12/50
2294/2294 [==============================] - 0s - loss: 0.2263 - ac

2294/2294 [==============================] - 0s - loss: 0.1269 - acc: 0.9599     
Epoch 38/50
2294/2294 [==============================] - 0s - loss: 0.1203 - acc: 0.9582     
Epoch 39/50
2294/2294 [==============================] - 0s - loss: 0.1199 - acc: 0.9586     
Epoch 40/50
2294/2294 [==============================] - 0s - loss: 0.1170 - acc: 0.9603     
Epoch 41/50
2294/2294 [==============================] - 0s - loss: 0.1177 - acc: 0.9586     
Epoch 42/50
2294/2294 [==============================] - 0s - loss: 0.1076 - acc: 0.9656     
Epoch 43/50
2294/2294 [==============================] - 0s - loss: 0.1134 - acc: 0.9625     
Epoch 44/50
2294/2294 [==============================] - 0s - loss: 0.1034 - acc: 0.9647     
Epoch 45/50
2294/2294 [==============================] - 0s - loss: 0.1093 - acc: 0.9643     
Epoch 46/50
2294/2294 [==============================] - 0s - loss: 0.0990 - acc: 0.9660     
Epoch 47/50
2294/2294 [==============================] - 0s - loss: 0.09

2294/2294 [==============================] - 0s - loss: 0.1616 - acc: 0.9377     
Epoch 21/50
2294/2294 [==============================] - 0s - loss: 0.1577 - acc: 0.9420     
Epoch 22/50
2294/2294 [==============================] - 0s - loss: 0.1598 - acc: 0.9451     
Epoch 23/50
2294/2294 [==============================] - 0s - loss: 0.1533 - acc: 0.9442     
Epoch 24/50
2294/2294 [==============================] - 0s - loss: 0.1456 - acc: 0.9459     
Epoch 25/50
2294/2294 [==============================] - 0s - loss: 0.1486 - acc: 0.9464     
Epoch 26/50
2294/2294 [==============================] - 0s - loss: 0.1452 - acc: 0.9477     
Epoch 27/50
2294/2294 [==============================] - 0s - loss: 0.1359 - acc: 0.9512     
Epoch 28/50
2294/2294 [==============================] - 0s - loss: 0.1379 - acc: 0.9499     
Epoch 29/50
2294/2294 [==============================] - 0s - loss: 0.1295 - acc: 0.9590     
Epoch 30/50
2294/2294 [==============================] - 0s - loss: 0.12

# CNN4: 32 conv 3x3 - 64 conv 3x3 - max pool 2x2 - 1024 fc

In [8]:
folds = 10
kfold = StratifiedKFold(n_splits=folds, shuffle=True)
cv_accuracies = []
cv_precision = []
cv_recall = []
cv_f1_score = []

for index, (train_indices, val_indices) in enumerate(kfold.split(tactile_images, labels)):
    print("Training fold " + str(index + 1) + "/" + str(folds))

    # split data
    images_train, images_val = tactile_images[train_indices], tactile_images[val_indices]
    labels_train, labels_val = labels_cat[train_indices], labels_cat[val_indices]  
    
    # build model
    epochs = 50
    batch = 32
        
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(FINGERS, TACTILE_IMAGE_ROWS, TACTILE_IMAGE_COLS), 
                     data_format='channels_first'))
    model.add(Activation('relu'))
    
    model.add(Conv2D(64, (3, 3), data_format='channels_first'))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(2,2), data_format='channels_first'))
    
    model.add(Flatten())
    
    model.add(Dense(1024))
    model.add(Activation('relu'))
    
    model.add(Dense(2, activation='softmax'))

    # compile and train
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=0.0001),
                  metrics=['accuracy'])
    model.fit(images_train, labels_train, epochs=epochs, batch_size=batch, verbose=1)
    
    # evaluate
    print("# # # Evaluating cv-fold...")
    scores = model.evaluate(images_val, labels_val, verbose=1)
    cv_accuracies.append(scores[1] * 100)
    
    predictions = model.predict(images_val)
    [precision, recall, f1_score, _] = precision_recall_fscore_support(np.argmax(labels_val, axis=1), 
                                                                          np.argmax(predictions, axis=1), 
                                                                          average='binary', pos_label=1)
    
    cv_precision.append(precision * 100)
    cv_recall.append(recall * 100)
    cv_f1_score.append(f1_score * 100)
    
    print("\naccuracy:", scores[1] * 100)
    print("precision:", precision * 100)
    print("recall:", recall * 100)
    print("f1_score:", f1_score * 100)
    
print("\nFinal cross-validation score:")
print(np.mean(cv_accuracies), "+/-", np.std(cv_accuracies))
print(np.mean(cv_precision), "+/-", np.std(cv_precision))
print(np.mean(cv_recall), "+/-", np.std(cv_recall))
print(np.mean(cv_f1_score), "+/-", np.std(cv_f1_score))

# IROS
#91.3694610724 +/- 1.10704031306
#91.4202624677 +/- 2.85125701535
#91.5551181102 +/- 2.56548519514
#91.418870124 +/- 1.01969081695

# cero2lesscontact - D7
#90.58468414107573 +/- 2.196870479476486
#91.62750439474084 +/- 3.705821039937615
#89.67950295275591 +/- 4.0108532365467875
#90.52437829974777 +/- 2.138180699338776

Training fold 1/10
Epoch 1/50
2294/2294 [==============================] - 1s - loss: 0.4834 - acc: 0.7877     
Epoch 2/50
2294/2294 [==============================] - 0s - loss: 0.3719 - acc: 0.8344     
Epoch 3/50
2294/2294 [==============================] - 0s - loss: 0.3356 - acc: 0.8505     
Epoch 4/50
2294/2294 [==============================] - 0s - loss: 0.3118 - acc: 0.8622     
Epoch 5/50
2294/2294 [==============================] - 0s - loss: 0.2923 - acc: 0.8771     
Epoch 6/50
2294/2294 [==============================] - 0s - loss: 0.2802 - acc: 0.8758     
Epoch 7/50
2294/2294 [==============================] - 0s - loss: 0.2731 - acc: 0.8862     
Epoch 8/50
2294/2294 [==============================] - 0s - loss: 0.2601 - acc: 0.8954     
Epoch 9/50
2294/2294 [==============================] - 0s - loss: 0.2533 - acc: 0.8971     
Epoch 10/50
2294/2294 [==============================] - 0s - loss: 0.2391 - acc: 0.9015     
Epoch 11/50
2294/2294 [===========================

2294/2294 [==============================] - 0s - loss: 0.1268 - acc: 0.9568     
Epoch 37/50
2294/2294 [==============================] - 0s - loss: 0.1168 - acc: 0.9629     
Epoch 38/50
2294/2294 [==============================] - 0s - loss: 0.1206 - acc: 0.9612     
Epoch 39/50
2294/2294 [==============================] - 0s - loss: 0.1260 - acc: 0.9525     
Epoch 40/50
2294/2294 [==============================] - 0s - loss: 0.1105 - acc: 0.9647     
Epoch 41/50
2294/2294 [==============================] - 0s - loss: 0.1112 - acc: 0.9629     
Epoch 42/50
2294/2294 [==============================] - 0s - loss: 0.1067 - acc: 0.9612     
Epoch 43/50
2294/2294 [==============================] - 0s - loss: 0.1041 - acc: 0.9682     
Epoch 44/50
2294/2294 [==============================] - 0s - loss: 0.1030 - acc: 0.9651     
Epoch 45/50
2294/2294 [==============================] - 0s - loss: 0.1003 - acc: 0.9638     
Epoch 46/50
2294/2294 [==============================] - 0s - loss: 0.09

2294/2294 [==============================] - 0s - loss: 0.1907 - acc: 0.9316     
Epoch 20/50
2294/2294 [==============================] - 0s - loss: 0.1887 - acc: 0.9255     
Epoch 21/50
2294/2294 [==============================] - 0s - loss: 0.1848 - acc: 0.9320     
Epoch 22/50
2294/2294 [==============================] - 0s - loss: 0.1785 - acc: 0.9311     
Epoch 23/50
2294/2294 [==============================] - 0s - loss: 0.1747 - acc: 0.9346     
Epoch 24/50
2294/2294 [==============================] - 0s - loss: 0.1685 - acc: 0.9403     
Epoch 25/50
2294/2294 [==============================] - 0s - loss: 0.1718 - acc: 0.9372     
Epoch 26/50
2294/2294 [==============================] - 0s - loss: 0.1646 - acc: 0.9442     
Epoch 27/50
2294/2294 [==============================] - 0s - loss: 0.1636 - acc: 0.9403     
Epoch 28/50
2294/2294 [==============================] - 0s - loss: 0.1492 - acc: 0.9477     
Epoch 29/50
2294/2294 [==============================] - 0s - loss: 0.15

2294/2294 [==============================] - 0s - loss: 0.3575 - acc: 0.8448     
Epoch 3/50
2294/2294 [==============================] - 0s - loss: 0.3285 - acc: 0.8618     
Epoch 4/50
2294/2294 [==============================] - 0s - loss: 0.3110 - acc: 0.8727     
Epoch 5/50
2294/2294 [==============================] - 0s - loss: 0.2907 - acc: 0.8810     
Epoch 6/50
2294/2294 [==============================] - 0s - loss: 0.2766 - acc: 0.8845     
Epoch 7/50
2294/2294 [==============================] - 0s - loss: 0.2676 - acc: 0.8963     
Epoch 8/50
2294/2294 [==============================] - 0s - loss: 0.2579 - acc: 0.8928     
Epoch 9/50
2294/2294 [==============================] - 0s - loss: 0.2580 - acc: 0.8941     
Epoch 10/50
2294/2294 [==============================] - 0s - loss: 0.2435 - acc: 0.8963     
Epoch 11/50
2294/2294 [==============================] - 0s - loss: 0.2404 - acc: 0.9045     
Epoch 12/50
2294/2294 [==============================] - 0s - loss: 0.2284 - ac

2294/2294 [==============================] - 0s - loss: 0.1206 - acc: 0.9621     
Epoch 38/50
2294/2294 [==============================] - 0s - loss: 0.1146 - acc: 0.9634     
Epoch 39/50
2294/2294 [==============================] - 0s - loss: 0.1115 - acc: 0.9647     
Epoch 40/50
2294/2294 [==============================] - 0s - loss: 0.1082 - acc: 0.9686     
Epoch 41/50
2294/2294 [==============================] - 0s - loss: 0.1053 - acc: 0.9647     
Epoch 42/50
2294/2294 [==============================] - 0s - loss: 0.1058 - acc: 0.9664     
Epoch 43/50
2294/2294 [==============================] - 0s - loss: 0.1086 - acc: 0.9643     
Epoch 44/50
2294/2294 [==============================] - 0s - loss: 0.0988 - acc: 0.9673     
Epoch 45/50
2294/2294 [==============================] - 0s - loss: 0.0967 - acc: 0.9708     
Epoch 46/50
2294/2294 [==============================] - 0s - loss: 0.0937 - acc: 0.9673     
Epoch 47/50
2294/2294 [==============================] - 0s - loss: 0.09

2294/2294 [==============================] - 0s - loss: 0.1844 - acc: 0.9350     
Epoch 21/50
2294/2294 [==============================] - 0s - loss: 0.1759 - acc: 0.9359     
Epoch 22/50
2294/2294 [==============================] - 0s - loss: 0.1731 - acc: 0.9359     
Epoch 23/50
2294/2294 [==============================] - 0s - loss: 0.1624 - acc: 0.9403     
Epoch 24/50
2294/2294 [==============================] - 0s - loss: 0.1627 - acc: 0.9394     
Epoch 25/50
2294/2294 [==============================] - 0s - loss: 0.1536 - acc: 0.9455     
Epoch 26/50
2294/2294 [==============================] - 0s - loss: 0.1524 - acc: 0.9455     
Epoch 27/50
2294/2294 [==============================] - 0s - loss: 0.1541 - acc: 0.9473     
Epoch 28/50
2294/2294 [==============================] - 0s - loss: 0.1465 - acc: 0.9499     
Epoch 29/50
2294/2294 [==============================] - 0s - loss: 0.1406 - acc: 0.9538     
Epoch 30/50
2294/2294 [==============================] - 0s - loss: 0.13

# CNN5: 32 conv 3x3 - 64 conv 3x3 - 1024 fc

In [9]:
folds = 10
kfold = StratifiedKFold(n_splits=folds, shuffle=True)
cv_accuracies = []
cv_precision = []
cv_recall = []
cv_f1_score = []

for index, (train_indices, val_indices) in enumerate(kfold.split(tactile_images, labels)):
    print("Training fold " + str(index + 1) + "/" + str(folds))

    # split data
    images_train, images_val = tactile_images[train_indices], tactile_images[val_indices]
    labels_train, labels_val = labels_cat[train_indices], labels_cat[val_indices]  
    
    # build model
    epochs = 50
    batch = 32
        
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(FINGERS, TACTILE_IMAGE_ROWS, TACTILE_IMAGE_COLS), 
                     data_format='channels_first'))
    model.add(Activation('relu'))
    
    model.add(Conv2D(64, (3, 3), data_format='channels_first'))
    model.add(Activation('relu'))
    
    model.add(Flatten())
    
    model.add(Dense(1024))
    model.add(Activation('relu'))
    
    model.add(Dense(2, activation='softmax'))

    # compile and train
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=0.0001),
                  metrics=['accuracy'])
    model.fit(images_train, labels_train, epochs=epochs, batch_size=batch, verbose=1)
    
    # evaluate
    print("# # # Evaluating cv-fold...")
    scores = model.evaluate(images_val, labels_val, verbose=1)
    cv_accuracies.append(scores[1] * 100)
    
    predictions = model.predict(images_val)
    [precision, recall, f1_score, _] = precision_recall_fscore_support(np.argmax(labels_val, axis=1), 
                                                                          np.argmax(predictions, axis=1), 
                                                                          average='binary', pos_label=1)
    
    cv_precision.append(precision * 100)
    cv_recall.append(recall * 100)
    cv_f1_score.append(f1_score * 100)
    
    print("\naccuracy:", scores[1] * 100)
    print("precision:", precision * 100)
    print("recall:", recall * 100)
    print("f1_score:", f1_score * 100)
    
print("\nFinal cross-validation score:")
print(np.mean(cv_accuracies), "+/-", np.std(cv_accuracies))
print(np.mean(cv_precision), "+/-", np.std(cv_precision))
print(np.mean(cv_recall), "+/-", np.std(cv_recall))
print(np.mean(cv_f1_score), "+/-", np.std(cv_f1_score))

# IROS
#91.643662054 +/- 1.20378746845
#91.9409073415 +/- 3.42026853616
#91.6326279528 +/- 3.42843528144
#91.6697740438 +/- 1.11241041534

# cero2lesscontact - D7
#91.87957380435267 +/- 2.2518199714590263
#91.69321090163643 +/- 2.258127232146895
#92.26070374015748 +/- 4.586570981643416
#91.89154490421365 +/- 2.4707468460319943

Training fold 1/10
Epoch 1/50
2294/2294 [==============================] - 1s - loss: 0.4408 - acc: 0.7916     
Epoch 2/50
2294/2294 [==============================] - 0s - loss: 0.3246 - acc: 0.8653     
Epoch 3/50
2294/2294 [==============================] - 0s - loss: 0.2927 - acc: 0.8727     
Epoch 4/50
2294/2294 [==============================] - 0s - loss: 0.2678 - acc: 0.8915     
Epoch 5/50
2294/2294 [==============================] - 0s - loss: 0.2482 - acc: 0.9002     
Epoch 6/50
2294/2294 [==============================] - 0s - loss: 0.2408 - acc: 0.9037     
Epoch 7/50
2294/2294 [==============================] - 0s - loss: 0.2330 - acc: 0.9058     
Epoch 8/50
2294/2294 [==============================] - 0s - loss: 0.2188 - acc: 0.9119     
Epoch 9/50
2294/2294 [==============================] - 0s - loss: 0.2115 - acc: 0.9133     
Epoch 10/50
2294/2294 [==============================] - 0s - loss: 0.2117 - acc: 0.9176     
Epoch 11/50
2294/2294 [===========================

2294/2294 [==============================] - 0s - loss: 0.0710 - acc: 0.9773     
Epoch 37/50
2294/2294 [==============================] - 0s - loss: 0.0723 - acc: 0.9795     
Epoch 38/50
2294/2294 [==============================] - 0s - loss: 0.0688 - acc: 0.9765     
Epoch 39/50
2294/2294 [==============================] - 0s - loss: 0.0762 - acc: 0.9712     
Epoch 40/50
2294/2294 [==============================] - 0s - loss: 0.0639 - acc: 0.9804     
Epoch 41/50
2294/2294 [==============================] - 0s - loss: 0.0676 - acc: 0.9773     
Epoch 42/50
2294/2294 [==============================] - 0s - loss: 0.0577 - acc: 0.9830     
Epoch 43/50
2294/2294 [==============================] - ETA: 0s - loss: 0.0519 - acc: 0.987 - 0s - loss: 0.0523 - acc: 0.9869     
Epoch 44/50
2294/2294 [==============================] - 0s - loss: 0.0547 - acc: 0.9843     
Epoch 45/50
2294/2294 [==============================] - 0s - loss: 0.0554 - acc: 0.9843     
Epoch 46/50
2294/2294 [===========

2294/2294 [==============================] - 0s - loss: 0.1417 - acc: 0.9503     
Epoch 19/50
2294/2294 [==============================] - 0s - loss: 0.1337 - acc: 0.9499     
Epoch 20/50
2294/2294 [==============================] - 0s - loss: 0.1242 - acc: 0.9608     
Epoch 21/50
2294/2294 [==============================] - 0s - loss: 0.1219 - acc: 0.9586     
Epoch 22/50
2294/2294 [==============================] - 0s - loss: 0.1274 - acc: 0.9534     
Epoch 23/50
2294/2294 [==============================] - 0s - loss: 0.1100 - acc: 0.9638     
Epoch 24/50
2294/2294 [==============================] - 0s - loss: 0.1044 - acc: 0.9647     
Epoch 25/50
2294/2294 [==============================] - 0s - loss: 0.1046 - acc: 0.9625     
Epoch 26/50
2294/2294 [==============================] - 0s - loss: 0.1064 - acc: 0.9664     
Epoch 27/50
2294/2294 [==============================] - 0s - loss: 0.0942 - acc: 0.9673     
Epoch 28/50
2294/2294 [==============================] - 0s - loss: 0.08

Epoch 1/50
2294/2294 [==============================] - 1s - loss: 0.4111 - acc: 0.8252     
Epoch 2/50
2294/2294 [==============================] - 0s - loss: 0.3136 - acc: 0.8675     
Epoch 3/50
2294/2294 [==============================] - 0s - loss: 0.2825 - acc: 0.8784     
Epoch 4/50
2294/2294 [==============================] - 0s - loss: 0.2554 - acc: 0.8997     
Epoch 5/50
2294/2294 [==============================] - 0s - loss: 0.2437 - acc: 0.9037     
Epoch 6/50
2294/2294 [==============================] - 0s - loss: 0.2260 - acc: 0.9115     
Epoch 7/50
2294/2294 [==============================] - 0s - loss: 0.2258 - acc: 0.9080     
Epoch 8/50
2294/2294 [==============================] - 0s - loss: 0.2132 - acc: 0.9146     
Epoch 9/50
2294/2294 [==============================] - 0s - loss: 0.1978 - acc: 0.9228     
Epoch 10/50
2294/2294 [==============================] - 0s - loss: 0.1895 - acc: 0.9281     
Epoch 11/50
2294/2294 [==============================] - 0s - loss: 0

2294/2294 [==============================] - 0s - loss: 0.0699 - acc: 0.9778     
Epoch 36/50
2294/2294 [==============================] - 0s - loss: 0.0635 - acc: 0.9821     
Epoch 37/50
2294/2294 [==============================] - 0s - loss: 0.0673 - acc: 0.9786     
Epoch 38/50
2294/2294 [==============================] - 0s - loss: 0.0634 - acc: 0.9839     
Epoch 39/50
2294/2294 [==============================] - 0s - loss: 0.0625 - acc: 0.9804     
Epoch 40/50
2294/2294 [==============================] - 0s - loss: 0.0603 - acc: 0.9843     
Epoch 41/50
2294/2294 [==============================] - 0s - loss: 0.0564 - acc: 0.9813     
Epoch 42/50
2294/2294 [==============================] - 0s - loss: 0.0540 - acc: 0.9861     
Epoch 43/50
2294/2294 [==============================] - 0s - loss: 0.0483 - acc: 0.9865     
Epoch 44/50
2294/2294 [==============================] - 0s - loss: 0.0501 - acc: 0.9869     
Epoch 45/50
2294/2294 [==============================] - 0s - loss: 0.05

2294/2294 [==============================] - 0s - loss: 0.1416 - acc: 0.9481     
Epoch 19/50
2294/2294 [==============================] - 0s - loss: 0.1381 - acc: 0.9525     
Epoch 20/50
2294/2294 [==============================] - 0s - loss: 0.1330 - acc: 0.9494     
Epoch 21/50
2294/2294 [==============================] - 0s - loss: 0.1251 - acc: 0.9577     
Epoch 22/50
2294/2294 [==============================] - 0s - loss: 0.1260 - acc: 0.9577     
Epoch 23/50
2294/2294 [==============================] - 0s - loss: 0.1163 - acc: 0.9621     
Epoch 24/50
2294/2294 [==============================] - 0s - loss: 0.1200 - acc: 0.9625     
Epoch 25/50
2294/2294 [==============================] - 0s - loss: 0.1125 - acc: 0.9590     
Epoch 26/50
2294/2294 [==============================] - 0s - loss: 0.1089 - acc: 0.9629     
Epoch 27/50
2294/2294 [==============================] - 0s - loss: 0.0995 - acc: 0.9682     
Epoch 28/50
2294/2294 [==============================] - 0s - loss: 0.09